Package Installations

In [1]:
import random
import copy
import time
import pandas as pd
import numpy as np
import gc
import re
import torch as t

#import spacy
from tqdm import tqdm_notebook, tnrange
from tqdm.auto import tqdm

tqdm.pandas(desc='Progress')
from collections import Counter

from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable
import os 

# cross validation and metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

from sklearn.preprocessing import StandardScaler
from multiprocessing import  Pool
from functools import partial
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt

In [2]:
!pip install kaggle

In [3]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 5.1 MB/s 
     |████████████████████████████████| 77 kB 7.7 MB/s 
     |████████████████████████████████| 596 kB 63.0 MB/s 
     |████████████████████████████████| 895 kB 57.9 MB/s 
     |████████████████████████████████| 6.6 MB 42.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
!pip install -U torchtext==0.8.0

     |████████████████████████████████| 6.9 MB 4.6 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0


In [5]:
import tensorflow as tf
import torch
import pandas as pd
import numpy as np
import random
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from tqdm.notebook import tqdm
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import transformers
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [6]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
seed_val = 500
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

Kaggle Dataset

In [8]:
# Create the kaggle directory and read the uploaded kaggle.json file
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [9]:
!chmod 600 /root/.kaggle/kaggle.json

In [10]:
# Download dataset
!kaggle datasets download -d clmentbisaillon/fake-and-real-news-dataset

 98% 40.0M/41.0M [00:00<00:00, 141MB/s]
100% 41.0M/41.0M [00:00<00:00, 128MB/s]


In [11]:
# Unzip folder in Colab content folder
!unzip /content/fake-and-real-news-dataset.zip

Archive:  /content/fake-and-real-news-dataset.zip
  inflating: Fake.csv                
  inflating: True.csv                


In [12]:
df1 = pd.read_csv("Fake.csv")
df2 = pd.read_csv("True.csv")
df1["y"] = ["False"]*len(df1)
df2["y"] = ["True"]*len(df2)

In [13]:
frames = [df1, df2]
corpus = pd.concat(frames, ignore_index=True)

In [14]:
corpus["x"] = corpus["title"]

In [15]:
y_encoder = LabelEncoder()
corpus['y'] = y_encoder.fit_transform(corpus['y'])

In [16]:
corpus = corpus[["x", "y"]]

In [17]:
corpus.drop_duplicates(inplace=True)
corpus

,x,y
0,Donald Trump Sends Out Embarrassing New Year’...,0
1,Drunk Bragging Trump Staffer Started Russian ...,0
2,Sheriff David Clarke Becomes An Internet Joke...,0
3,Trump Is So Obsessed He Even Has Obama’s Name...,0
4,Pope Francis Just Called Out Donald Trump Dur...,0
...,...,...
44892,North Korea shipments to Syria chemical arms a...,1
44894,LexisNexis withdrew two products from Chinese ...,1
44895,Minsk cultural hub becomes haven from authorities,1
44896,Vatican upbeat on possibility of Pope Francis ...,1


**Twitter Dataset**

In [18]:
twitter_real_df = pd.read_csv('/content/real_news_tweets.csv')
twitter_real_df

,Unnamed: 0,Datetime,Tweet Id,Text,Username
0,0,2022-04-19 08:35:25+00:00,1516334641610989571,Half-a-million Ukrainians have fled to the Lvi...,AP
1,1,2022-04-19 07:42:46+00:00,1516321393281015809,Afghan police say explosions targeting educati...,AP
2,2,2022-04-19 07:13:55+00:00,1516314133666349058,Russian forces have attacked along a broad fro...,AP
3,3,2022-04-19 06:40:38+00:00,1516305757674819584,There's stark public divide in Iran over Russi...,AP
4,4,2022-04-19 06:26:26+00:00,1516302180772364288,There's stark public divide in Iran over Russi...,AP
...,...,...,...,...,...
4995,4995,2022-02-24 00:13:22+00:00,1496639353820430337,The soldiers on the front line of the Ukraine-...,BBCWorld
4996,4996,2022-02-23 23:45:08+00:00,1496632246459158531,What's Putin's next move?,BBCWorld
4997,4997,2022-02-23 23:29:06+00:00,1496628212880875520,"Virgin Hyperloop, a futuristic train service e...",BBCWorld
4998,4998,2022-02-23 23:14:06+00:00,1496624437243879429,Metaverse app allows kids into virtual strip c...,BBCWorld


In [19]:
twitter_fake_df = pd.read_csv('/content/fake_news_tweets.csv')
twitter_fake_df

,Unnamed: 0,Datetime,Tweet Id,Text,Username
0,0,2022-04-19 08:30:02+00:00,1516333287744102405,Grandpa Surprisingly Willing To Talk About Man...,TheOnion
1,1,2022-04-19 06:30:01+00:00,1516303084628099076,Nation Attempts To Fall Asleep By Doing Little...,TheOnion
2,2,2022-04-19 05:00:01+00:00,1516280436334473221,"When it comes to pip blips, Megan Fox takes th...",TheOnion
3,3,2022-04-19 04:30:01+00:00,1516272886314672130,Longtime Sleepytime Tea Addict Has To Use 6 Ba...,TheOnion
4,4,2022-04-19 03:15:00+00:00,1516254005869096968,The Onion: Now on another part of your phone. ...,TheOnion
...,...,...,...,...,...
4995,4995,2021-10-02 14:00:00+00:00,1444301140703780868,Boss Warns All New Employees He Will Fire Them...,TheBabylonBee
4996,4996,2021-10-02 13:00:02+00:00,1444286051204403205,10 Tips For Making Sure Your Kids Are Terrifie...,TheBabylonBee
4997,4997,2021-10-02 12:21:32+00:00,1444276362865479696,Biden Says We Can Afford $3.5T Bill Because Ch...,TheBabylonBee
4998,4998,2021-10-02 02:00:00+00:00,1444119947274334211,Fly That Landed On Mike Pence During Debate Re...,TheBabylonBee


In [20]:
twitter_real_df['x'] = twitter_real_df['Text']
twitter_fake_df['x'] = twitter_fake_df['Text']

In [21]:
twitter_real_df['y'] = ['True'] * 5000
twitter_real_df['y'] = y_encoder.transform(twitter_real_df['y'])

In [22]:
twitter_fake_df['y'] = ['False'] * 5000
twitter_fake_df['y'] = y_encoder.transform(twitter_fake_df['y'])

In [23]:
twitter_real_df = twitter_real_df[["x", "y"]]
twitter_real_df.drop_duplicates(inplace=True)
twitter_real_df

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,x,y
0,Half-a-million Ukrainians have fled to the Lvi...,1
1,Afghan police say explosions targeting educati...,1
2,Russian forces have attacked along a broad fro...,1
3,There's stark public divide in Iran over Russi...,1
4,There's stark public divide in Iran over Russi...,1
...,...,...
4995,The soldiers on the front line of the Ukraine-...,1
4996,What's Putin's next move?,1
4997,"Virgin Hyperloop, a futuristic train service e...",1
4998,Metaverse app allows kids into virtual strip c...,1


In [24]:
twitter_fake_df = twitter_fake_df[["x", "y"]]
twitter_fake_df.drop_duplicates(inplace=True)
twitter_fake_df

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,x,y
0,Grandpa Surprisingly Willing To Talk About Man...,0
1,Nation Attempts To Fall Asleep By Doing Little...,0
2,"When it comes to pip blips, Megan Fox takes th...",0
3,Longtime Sleepytime Tea Addict Has To Use 6 Ba...,0
4,The Onion: Now on another part of your phone. ...,0
...,...,...
4995,Boss Warns All New Employees He Will Fire Them...,0
4996,10 Tips For Making Sure Your Kids Are Terrifie...,0
4997,Biden Says We Can Afford $3.5T Bill Because Ch...,0
4998,Fly That Landed On Mike Pence During Debate Re...,0


In [25]:
from sklearn import model_selection
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [26]:
frame1 = [twitter_real_df, twitter_fake_df]
corpus_tweet = pd.concat(frame1)
corpus_tweet

,x,y
0,Half-a-million Ukrainians have fled to the Lvi...,1
1,Afghan police say explosions targeting educati...,1
2,Russian forces have attacked along a broad fro...,1
3,There's stark public divide in Iran over Russi...,1
4,There's stark public divide in Iran over Russi...,1
...,...,...
4995,Boss Warns All New Employees He Will Fire Them...,0
4996,10 Tips For Making Sure Your Kids Are Terrifie...,0
4997,Biden Says We Can Afford $3.5T Bill Because Ch...,0
4998,Fly That Landed On Mike Pence During Debate Re...,0


In [27]:
x_train =corpus['x']
y_train = corpus['y']
x_test = corpus_tweet['x']
y_test = corpus_tweet['y']

In [28]:
embed_size = 300 # how big is each word vector
max_features = 120000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 750 # max number of words in a question to use
batch_size = 512 # how many samples to process at once
n_epochs = 5 # how many times to iterate over all samples
n_splits = 5 # Number of K-fold Splits
SEED = 10
debug = 0

In [29]:
## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(x_train))
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

## Pad the sentences 
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test= pad_sequences(x_test, maxlen=maxlen)

In [30]:
print("X_test: ",x_train, x_train.shape, x_train.shape[1])
print("X_test: ",x_test)

X_test:  [[    0     0     0 ...    43    15  1771]
 [    0     0     0 ...    87  1806   379]
 [    0     0     0 ...  7193     8 11526]
 ...
 [    0     0     0 ...  6253    22  2434]
 [    0     0     0 ...  3112  3557    38]
 [    0     0     0 ...     4    87  2404]] (38729, 750) 750
X_test:  [[   0    0    0 ...   15   11  582]
 [   0    0    0 ...   11 3951 4154]
 [   0    0    0 ...   17  577 1363]
 ...
 [   0    0    0 ...   11 1051 1539]
 [   0    0    0 ...  990    4 1566]
 [   0    0    0 ... 1426    7 1876]]


In [31]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip
!ls -lat

--2022-04-21 15:36:09--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-04-21 15:36:10--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-04-21 15:36:10--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [32]:
## FUNCTIONS TAKEN FROM https://www.kaggle.com/gmhost/gru-capsule

def load_glove(word_index):
    EMBEDDING_FILE = '/content/glove.6B.300d.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')[:300]
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))
    
    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = -0.005838499,0.48782197
    embed_size = all_embs.shape[1]

    nb_words = min(max_features, len(word_index)+1)
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: 
            embedding_matrix[i] = embedding_vector
        else:
            embedding_vector = embeddings_index.get(word.capitalize())
            if embedding_vector is not None: 
                embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [33]:
if debug:
    embedding_matrix = np.random.randn(120000,300)
else:
    embedding_matrix = load_glove(tokenizer.word_index)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [34]:
np.shape(embedding_matrix)

(31803, 300)

BiLSTM

In [35]:
import torch

In [36]:
class BiLSTM(nn.Module):
    
    def __init__(self):
        super(BiLSTM, self).__init__()
        self.hidden_size = 64
        drp = 0.3
        n_classes = 2
        self.embedding = nn.Embedding(max_features, embed_size)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
        self.lstm = nn.LSTM(embed_size, self.hidden_size, bidirectional=True, batch_first=True)
        self.linear = nn.Linear(self.hidden_size*4 , 64)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(drp)
        self.out = nn.Linear(64, n_classes)


    def forward(self, x):
        #rint(x.size())
        h_embedding = self.embedding(x)
        #_embedding = torch.squeeze(torch.unsqueeze(h_embedding, 0))
        h_lstm, _ = self.lstm(h_embedding)
        avg_pool = torch.mean(h_lstm, 1)
        max_pool, _ = torch.max(h_lstm, 1)
        conc = torch.cat(( avg_pool, max_pool), 1)
        conc = self.relu(self.linear(conc))
        conc = self.dropout(conc)
        out = self.out(conc)
        return out

In [37]:
n_epochs = 10
model = BiLSTM()
loss_fn = nn.CrossEntropyLoss(reduction='sum')
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
model.cuda()

# Load train and test in CUDA Memory
x_train2 = torch.tensor(x_train, dtype=torch.long).cuda()
y_train2 = torch.tensor(y_train.values, dtype=torch.long).cuda()
x_cv = torch.tensor(x_test, dtype=torch.long).cuda()
y_cv = torch.tensor(y_test.values, dtype=torch.long).cuda()

# Create Torch datasets
train = torch.utils.data.TensorDataset(x_train2, y_train2)
valid = torch.utils.data.TensorDataset(x_cv, y_cv)

# Create Data Loaders
train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=False)

train_loss = []
valid_loss = []

for epoch in range(n_epochs):
    start_time = time.time()
    # Set model to train configuration
    model.train()
    avg_loss = 0.  
    for i, (x_batch, y_batch) in enumerate(train_loader):
        # Predict/Forward Pass
        y_pred = model(x_batch)
        # Compute loss
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        avg_loss += loss.item() / len(train_loader)
    
    # Set model to validation configuration -Doesn't get trained here
    model.eval()        
    avg_val_loss = 0.
    val_preds = np.zeros((len(x_cv),2))
    
    for i, (x_batch, y_batch) in enumerate(valid_loader):
        y_pred = model(x_batch).detach()
        avg_val_loss += loss_fn(y_pred, y_batch).item() / len(valid_loader)
        # keep/store predictions
        val_preds[i * batch_size:(i+1) * batch_size] =F.softmax(y_pred).cpu().numpy()
    
    # Check Accuracy
    val_accuracy = sum(val_preds.argmax(axis=1)==y_test)/len(y_test)
    train_loss.append(avg_loss)
    valid_loss.append(avg_val_loss)
    elapsed_time = time.time() - start_time 
    print('Epoch {}/{} \t loss={:.4f} \t val_loss={:.4f}  \t val_acc={:.4f}  \t time={:.2f}s'.format(
                epoch + 1, n_epochs, avg_loss, avg_val_loss, val_accuracy, elapsed_time))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 1/10 	 loss=164.6099 	 val_loss=950.4117  	 val_acc=0.5232  	 time=15.55s
Epoch 2/10 	 loss=65.3229 	 val_loss=1207.9792  	 val_acc=0.5079  	 time=15.68s
Epoch 3/10 	 loss=50.0006 	 val_loss=1364.8319  	 val_acc=0.5149  	 time=16.72s
Epoch 4/10 	 loss=37.9129 	 val_loss=1541.6039  	 val_acc=0.5106  	 time=18.81s
Epoch 5/10 	 loss=30.0608 	 val_loss=1666.1304  	 val_acc=0.5035  	 time=16.61s
Epoch 6/10 	 loss=22.2756 	 val_loss=1875.4252  	 val_acc=0.5143  	 time=17.04s
Epoch 7/10 	 loss=16.6324 	 val_loss=2060.6814  	 val_acc=0.4928  	 time=17.60s
Epoch 8/10 	 loss=11.7290 	 val_loss=2189.0716  	 val_acc=0.5011  	 time=17.90s
Epoch 9/10 	 loss=7.4647 	 val_loss=2402.5360  	 val_acc=0.5121  	 time=17.41s
Epoch 10/10 	 loss=5.5315 	 val_loss=2606.4161  	 val_acc=0.5136  	 time=17.97s
